## **Libraries and Dependencies**

In [1]:
# Imports
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers, optimizers, applications
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.layers import RandomZoom, RandomRotation, RandomFlip, Rescaling, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from PIL import Image
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Input
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, LearningRateScheduler
from tensorflow.keras.applications import VGG16
from sklearn.model_selection import KFold
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Sequential
import os
import random
import pandas as pd
import cv2
import logging
import warnings
import json


# Suppress warnings from the logging module
logging.getLogger('tensorflow').setLevel(logging.ERROR)
warnings.filterwarnings("ignore", category=UserWarning)


## **Tensorflow Version**

In [4]:
# Tensorflow Version
print(tf.__version__)

2.10.0


## **GPU Checker**

In [5]:
# Check if any GPU devices are detected
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"GPUs detected: {len(gpus)}")
else:
    print("No GPU detected.")

GPUs detected: 1


In [2]:
# Suppress TensorFlow logging except for fatal errors.
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [3]:
# Set seed
SEED = 338424

# Global variables
IMG_SIZE = (128, 128)
BATCH_SIZE = 32
num_classes = 18 # Number of folders in dataset
AUTOTUNE = tf.data.AUTOTUNE

## **VGG-16 Dataset: Loading, Splitting, Shuffling, Caching**

In [8]:
# Load Dataset
dataset_dir = 'dataset/hagridset'
full_ds_vgg16 = tf.keras.utils.image_dataset_from_directory(
    dataset_dir,
    shuffle=True,
    seed=SEED,
    image_size=(IMG_SIZE),
    batch_size=BATCH_SIZE,
    label_mode='categorical'
)

# Split into training, validation, and test sets
train_ratio_vgg16 = 0.7
val_ratio_vgg16 = 0.2
test_ratio_vgg16 = 0.1

# Total length of the dataset
total_size_vgg16 = len(full_ds_vgg16)

# Compute indices for the splits
train_size_vgg16 = int(total_size_vgg16 * train_ratio_vgg16)
val_size_vgg16 = int(total_size_vgg16 * val_ratio_vgg16)
test_size_vgg16 = total_size_vgg16 - (train_size_vgg16 + val_size_vgg16)

# Split the dataset and shuffle
train_ds_vgg16 = full_ds_vgg16.take(train_size_vgg16).shuffle(train_size_vgg16, seed=SEED)
val_ds_vgg16 = full_ds_vgg16.skip(train_size_vgg16).take(val_size_vgg16).shuffle(val_size_vgg16, seed=SEED)
test_ds_vgg16 = full_ds_vgg16.skip(train_size_vgg16 + val_size_vgg16).shuffle(test_size_vgg16, seed=SEED)

# Cache the dataset in memory (or use a directory to store it on disk if necessary)
train_ds_vgg16 = full_ds_vgg16.take(train_size_vgg16).shuffle(train_size_vgg16, seed=SEED).cache().prefetch(buffer_size=AUTOTUNE)
val_ds_vgg16 = full_ds_vgg16.skip(train_size_vgg16).take(val_size_vgg16).shuffle(val_size_vgg16, seed=SEED).cache().prefetch(buffer_size=AUTOTUNE)
test_ds_vgg16 = full_ds_vgg16.skip(train_size_vgg16 + val_size_vgg16).cache().prefetch(buffer_size=AUTOTUNE)

# Count samples in each subset
def count_samples(dataset):
    sample_count = sum(1 for _ in dataset.unbatch())
    return sample_count

# Output the number of samples for each dataset
print(f'Using {count_samples(train_ds_vgg16)} samples in the Training set')
print(f'Using {count_samples(val_ds_vgg16)} samples in the Validation set')
print(f'Using {count_samples(test_ds_vgg16)} samples in the Test set')

Found 125912 files belonging to 18 classes.
Using 88128 samples in the Training set
Using 25184 samples in the Validation set
Using 12600 samples in the Test set


In [9]:
# Get class names
class_names = full_ds_vgg16.class_names
class_names

['call',
 'dislike',
 'fist',
 'four',
 'like',
 'mute',
 'ok',
 'one',
 'palm',
 'peace',
 'peace_inverted',
 'rock',
 'stop',
 'stop_inverted',
 'three',
 'three2',
 'two_up',
 'two_up_inverted']

#### **Dataset: Transfer Learning VGG-16 - Save**

In [10]:
path_to_save = 'savedDatasetTLVGG16'

tf.data.experimental.save(train_ds_vgg16, path_to_save + '/train')
tf.data.experimental.save(val_ds_vgg16, path_to_save + '/val')
tf.data.experimental.save(test_ds_vgg16, path_to_save + '/test')

In [4]:
# Load the dataset Grayscale Models were trained on
path_to_load_TLVGG16_dataset = 'savedDatasetTLVGG16'
train_ds_vgg16 = tf.data.experimental.load(path_to_load_TLVGG16_dataset + '/train')
val_ds_vgg16 = tf.data.experimental.load(path_to_load_TLVGG16_dataset + '/val')
test_ds_vgg16 = tf.data.experimental.load(path_to_load_TLVGG16_dataset + '/test')

## **Regularization Factors**

This code snippet defines the values for L1 and L2 regularization, which are both set to 0.01. It then creates an "Elastic Net Regularizer" that combines these L1 and L2 values to help prevent the model from overfitting by penalizing overly complex or large weight values in the model's learning process.

In [5]:
# Define L1 and L2 regularization factors
l1_factor = 0.01  # Example value
l2_factor = 0.01  # Example value

# Elastic Net Regularizer
elastic_net_regularizer = regularizers.l1_l2(l1=l1_factor, l2=l2_factor)

## **Callbacks: Learning Rate Scheduler and Early Stopping**

In [6]:
# Define a learning rate schedule
def lr_time_based_decay(epoch, lr):
    # This function adjusts the learning rate over each epoch based on the initial learning rate,
    # applying a decay factor that increases with the epoch number. It effectively reduces the 
    # learning rate over time, which can help in calibrating the model adjustments as it 
    # approaches a minimum in the loss surface.
    return lr * 1 / (1 + 0.01 * epoch)

# Define callbacks
callbacks = [
    # EarlyStopping prevents overfitting by stopping training when the validation loss 
    # has not improved for 3 consecutive epochs ('patience=3'). It also restores the 
    # weights of the model to those of the epoch with the best validation loss, ensuring 
    # the model retains the best learned features even if it starts to overfit afterward.
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    # LearningRateScheduler adjusts the learning rate according to the lr_time_based_decay function above.
    # It logs the new learning rate at the start of each epoch ('verbose=1'), helping to control
    # the step size of model updates, which can be crucial for reaching convergence efficiently.
    LearningRateScheduler(lr_time_based_decay, verbose=1)
]

## **Data Augmentation Layers**

In [7]:
# Data Augmentation
data_augmentation_layers = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.05),
])

---

# **Convolutional Neural Networks (CNN): Transfer Learning VGG-16**

---

## **CNN Model: Transfer Learning VGG-16**

In [15]:
# Define the base model
base_model = VGG16(
    weights='imagenet',  # Load weights pre-trained on ImageNet
    input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3),
    include_top=False
)
base_model.trainable = False  # Freeze the base model

# Define the inputs
inputs = Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3))

# Preprocessing input according to VGG16
x = preprocess_input(inputs)

# Pass the inputs through the base model
x = base_model(x, training=False)

# Add GlobalAveragePooling2D layer
x = GlobalAveragePooling2D()(x)

# Output layer
outputs = Dense(num_classes, activation='softmax')(x)

# Define the model
model_transfer_learning_vgg16 = Model(inputs, outputs)

# Compile the model
model_transfer_learning_vgg16.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Summary of the model
model_transfer_learning_vgg16.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 tf.__operators__.getitem (S  (None, 128, 128, 3)      0         
 licingOpLambda)                                                 
                                                                 
 tf.nn.bias_add (TFOpLambda)  (None, 128, 128, 3)      0         
                                                                 
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 global_average_pooling2d (G  (None, 512)              0         
 lobalAveragePooling2D)                                          
                                                                 
 dense_9 (Dense)             (None, 18)                9234  

### **CNN Model: Transfer Learning VGG-16 - Training**

In [16]:
# Train CNN Model
history_vgg16 = model_transfer_learning_vgg16.fit(
    train_ds_vgg16,
    validation_data=val_ds_vgg16,
    epochs=10,
    callbacks=callbacks
)


Epoch 1: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 1/10
2754/2754 [==============================] - 92s 33ms/step - loss: 2.3931 - accuracy: 0.4368 - val_loss: 1.6101 - val_accuracy: 0.5322 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.0009900990569281696.
Epoch 2/10
2754/2754 [==============================] - 81s 30ms/step - loss: 1.6405 - accuracy: 0.5213 - val_loss: 1.5788 - val_accuracy: 0.5395 - lr: 9.9010e-04

Epoch 3: LearningRateScheduler setting learning rate to 0.0009706853341092082.
Epoch 3/10
2754/2754 [==============================] - 83s 30ms/step - loss: 1.6218 - accuracy: 0.5249 - val_loss: 1.5807 - val_accuracy: 0.5400 - lr: 9.7069e-04

Epoch 4: LearningRateScheduler setting learning rate to 0.0009424129424128428.
Epoch 4/10
2754/2754 [==============================] - 86s 31ms/step - loss: 1.6114 - accuracy: 0.5267 - val_loss: 1.5777 - val_accuracy: 0.5406 - lr: 9.4241e-04

Epoch 5: LearningRateScheduler 

### **CNN Model: Transfer Learning VGG-16 - Save**

In [17]:
# Save Model: .json
# Saves the Model Architecture
for key in history_vgg16.history.keys():
    history_vgg16.history[key] = [float(i) for i in history_vgg16.history[key]]

# Write the JSON file
with open('json/tl_vgg16_model_shallow.json', 'w') as f:
    json.dump(history_vgg16.history, f)


# Save Model: .h5
# Saves the Model Weights and Configurations
model_transfer_learning_vgg16.save('h5/tl_vgg16_model_shallow.h5')

## **CNN Model: Transfer Learning VGG-16 Fine Tuning**

In [12]:
def build_transfer_model(input_shape, num_classes):
    # Load the VGG16 base model with ImageNet weights
    base_model = VGG16(
        weights='imagenet', 
        input_shape=input_shape, 
        include_top=False
    )

    # Unfreeze the last 8 layers for fine-tuning
    base_model.trainable = True  # Make the model trainable
    for layer in base_model.layers[:-2]:
        layer.trainable = False
    for layer in base_model.layers[-2:]:
        layer.trainable = True

    # Define the input layer
    inputs = Input(shape=input_shape)

    # Pass the input through the base model
    x = base_model(inputs, training=False)  # Use training=False to ensure that batchnorm layers do not update during forward pass

    # Flatten the output of the base model
    x = Flatten()(x)

    # Add the first dense layer
    x = Dense(1024, activation='relu', kernel_regularizer=elastic_net_regularizer)(x)
    x = Dropout(0.05)(x)

    # Add the second dense layer
    x = Dense(512, activation='relu', kernel_regularizer=elastic_net_regularizer)(x)
    x = Dropout(0.05)(x)

    # Add the output layer
    outputs = Dense(num_classes, activation='softmax', dtype='float32')(x)  # Use float32 for dtype to ensure precision

    # Construct the model
    model = Model(inputs=inputs, outputs=outputs, name="model_transfer_learning_vgg16_fine_tuned")

    # Compile the model
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
        loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
        metrics=['accuracy']
    )

    return model

# Constants
input_shape = (IMG_SIZE[0], IMG_SIZE[1], 3)
num_classes = 18  # Number of classes in the dataset

# Create the model
model_transfer_learning_vgg16_fine_tuned = build_transfer_model(input_shape, num_classes)

# Output the model summary
model_transfer_learning_vgg16_fine_tuned.summary()

Model: "model_transfer_learning_vgg16_fine_tuned"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten_2 (Flatten)         (None, 8192)              0         
                                                                 
 dense_6 (Dense)             (None, 1024)              8389632   
                                                                 
 dropout_4 (Dropout)         (None, 1024)              0         
                                                                 
 dense_7 (Dense)             (None, 512)               524800    
                                                                 
 dropout_5 (Dropout)      

### **CNN Model: Transfer Learning VGG-16 Fine Tuning - Training**

In [13]:
# Train CNN Model
history_vgg16_fine_tuned = model_transfer_learning_vgg16_fine_tuned.fit(
    train_ds_vgg16,
    validation_data=val_ds_vgg16,
    epochs=10,
    callbacks=callbacks
)


Epoch 1: LearningRateScheduler setting learning rate to 9.999999747378752e-05.
Epoch 1/10
2754/2754 [==============================] - 108s 39ms/step - loss: 137.1809 - accuracy: 0.6177 - val_loss: 2.5834 - val_accuracy: 0.7438 - lr: 1.0000e-04

Epoch 2: LearningRateScheduler setting learning rate to 9.900989848889853e-05.
Epoch 2/10
2754/2754 [==============================] - 90s 33ms/step - loss: 2.2385 - accuracy: 0.7438 - val_loss: 2.0146 - val_accuracy: 0.7886 - lr: 9.9010e-05

Epoch 3: LearningRateScheduler setting learning rate to 9.706852913094575e-05.
Epoch 3/10
2754/2754 [==============================] - 90s 33ms/step - loss: 1.9399 - accuracy: 0.7892 - val_loss: 1.8571 - val_accuracy: 0.8089 - lr: 9.7069e-05

Epoch 4: LearningRateScheduler setting learning rate to 9.424129000286236e-05.
Epoch 4/10
2754/2754 [==============================] - 89s 32ms/step - loss: 1.7782 - accuracy: 0.8175 - val_loss: 1.7663 - val_accuracy: 0.8193 - lr: 9.4241e-05

Epoch 5: LearningRateSch

### **CNN Model: Transfer Learning VGG-16 Fine Tuning - Save**

In [14]:
# Save Model: .json
# Saves the Model Architecture
for key in history_vgg16_fine_tuned.history.keys():
    history_vgg16_fine_tuned.history[key] = [float(i) for i in history_vgg16_fine_tuned.history[key]]

# Write the JSON file
with open('json/tl_vgg16_model_vgg16_fine_tuned.json', 'w') as f:
    json.dump(history_vgg16_fine_tuned.history, f)


# Save Model: .h5
# Saves the Model Weights and Configurations
model_transfer_learning_vgg16_fine_tuned.save('h5/tl_vgg16_model_fine_tuned.h5')